In [293]:
import pandas as pd 
import geopandas as gpd
pd.options.display.max_columns=1000
pd.options.display.max_rows=1000


In [300]:
def string_punc(test_str):

    punc = """!()-[]{};:'"\,<>./?@#$%^&*_~"""

    try:

        test_str = str(test_str).lower()
        # print(test_str)
        for ele in test_str:
            if ele in punc:
                test_str = test_str.replace(ele, "")
        processed_lst = []
        for word in test_str.split():
            if word not in ["the", "at", "in", "and", "of"]:
                processed_lst.append(word)

        test_str = " ".join(processed_lst)
        test_str = test_str.replace(" ", "")
        return test_str.lower()
    except:
        return "Null"

geo_df = gpd.read_file("../20_intermediate_files/10_HIFLD_campus_polygons.geojson")
master_df = pd.read_csv(
    "../00_source_data/College_Data/SLSV Master Campus Sheet - Master Sheet.csv",
    header=1,
)
print("Original Master:", len(master_df))

master_df["preprocessed_name"] = master_df["School Name"].apply(string_punc)
geo_df["preprocessed_name"] = geo_df["preprocessed_name"].apply(string_punc)
master_df=master_df.drop_duplicates()
master_df = master_df[
    ~master_df["preprocessed_name"].str.contains(
        "policy|law|phd|graduate|grad|health|medicine|medical"
        "|music|business|nursing|film|art|arts|design|commerce",
        regex=True,
    )
]
master_df=master_df[master_df['IPED ID']!='Hawaii Community College']
master_df=master_df[master_df['IPED ID']!='485263.00']
master_df=master_df[master_df['IPED ID']!='101949.00']
master_df = master_df[master_df['IPED ID'] != '155636.00']

#geo_df = geo_df[geo_df['preprocessed_name']!='stevenshenagercollege']
#geo_df = geo_df[geo_df['preprocessed_name']!='laureltechnicalinstitute']
#geo_df = geo_df[(geo_df['preprocessed_name']!='westerntechnicalcollege') &(geo_df['STATE']!='TX')]
geo_df = geo_df.loc[~geo_df.duplicated(['preprocessed_name', "STATE"])]

geo_df=geo_df[geo_df['UNIQUEID']!='459082']
geo_df=geo_df[geo_df['UNIQUEID']!='241128']

print("Updated Master:", len(master_df))

Original Master: 3114
Updated Master: 2838


In [301]:
geo_df

,UNIQUEID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,TELEPHONE,TYPE,STATUS,POPULATION,COUNTY,COUNTYFIPS,COUNTRY,NAICS_CODE,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,WEBSITE,TOT_ENROLL,TOT_EMP,SHELTERID,preprocessed_name,centroid_x_epsg_102005,centroid_y_epsg_102005,centroid_long,centroid_lat,geometry
0,105367,OTTAWA UNIVERSITY-PHOENIX,9414 NORTH 25TH AVENUE,PHOENIX,AZ,85021,NOT AVAILABLE,(602) 371-1188,2,A,923,MARICOPA,04013,USA,611310,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",www.ottawa.edu,2018-02-28,IMAGERY/OTHER,2019-01-24,www.ottawa.edu,809,114,NOT AVAILABLE,ottawauniversityphoenix,-1.486844e+06,-470622.131776,-112.113054,33.572125,"POLYGON ((-1486881.628 -470586.100, -1486793.2..."
1,110361,CALIFORNIA BAPTIST UNIVERSITY,8432 MAGNOLIA AVE,RIVERSIDE,CA,92504,3297,(877) 228-3615,2,A,12142,RIVERSIDE,06065,USA,611310,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",https://calbaptist.edu/map,2018-05-10,IMAGERY/OTHER,2019-01-24,www.calbaptist.edu,10486,1656,NOT AVAILABLE,californiabaptistuniversity,-1.960011e+06,-331557.164594,-117.426029,33.927877,"POLYGON ((-1960482.345 -331343.278, -1960078.2..."
2,111391,CALIFORNIA WESTERN SCHOOL OF LAW,225 CEDAR ST,SAN DIEGO,CA,92101,3046,(619) 239-0391,2,A,1021,SAN DIEGO,06073,USA,611310,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",https://www.cwsl.edu,2018-05-11,IMAGERY/OTHER,2019-01-24,https://www.cwsl.edu,802,219,NOT AVAILABLE,californiawesternschoollaw,-1.967108e+06,-467374.141459,-117.162558,32.721731,"POLYGON ((-1967135.570 -467340.979, -1967068.1..."
3,113698,DOMINICAN UNIVERSITY OF CALIFORNIA,50 ACACIA AVE,SAN RAFAEL,CA,94901,2298,(415) 457-4440,2,A,2395,MARIN,06041,USA,611310,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",https://www.dominican.edu/about/campus/mapsand...,2018-08-30,IMAGERY/OTHER,2019-01-24,www.dominican.edu,1779,616,NOT AVAILABLE,dominicanuniversitycalifornia,-2.284703e+06,221228.999296,-122.514107,37.980091,"MULTIPOLYGON (((-2284182.288 221224.380, -2284..."
4,192563,LIU BRENTWOOD,"MICHAEL J. GRANT CAMPUS, SUFFOLK COUNTY COMMUN...",BRENTWOOD,NY,11717,NOT AVAILABLE,(631) 287-8500,2,C,-999,SUFFOLK,36103,USA,611310,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",http://www.liu.edu/Brentwood/About/Visiting/Ca...,2018-12-19,IMAGERY/OTHER,2019-04-11,liu.edu/Brentwood,-999,-999,NOT AVAILABLE,liubrentwood,1.888610e+06,436153.557383,-73.272732,40.797865,"POLYGON ((1888955.082 435997.571, 1888956.510 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5391,11493801,GAVILAN COLLEGE - HOLLISTER SITE,365 FOURTH STREET,HOLLISTER,CA,95023,NOT AVAILABLE,NOT AVAILABLE,1,A,1361,SAN BENITO,06069,USA,611210,JUNIOR COLLEGES,https://www.gavilan.edu/about/hollister.php,2018-12-19,IMAGERY/OTHER,2019-02-12,http://www.gavilan.edu/hollister/index.html,1361,-999,NOT AVAILABLE,gavilancollegehollistersite,-2.225901e+06,73606.022079,-121.402782,36.852230,"POLYGON ((-2225915.052 73641.116, -2225870.144..."
5392,11493802,GAVILAN COLLEGE - MORGAN HILL CAMPUS,17060 MONTEREY ST.,MORGAN HILL,CA,95037,NOT AVAILABLE,NOT AVAILABLE,1,A,1323,SANTA CLARA,06085,USA,611210,JUNIOR COLLEGES,https://www.gavilan.edu/about/morgan_hill.php,2018-12-19,IMAGERY/OTHER,2019-02-12,http://www.gavilan.edu/maps/index.html,1323,-999,NOT AVAILABLE,gavilancollegemorganhillcampus,-2.238508e+06,108881.422727,-121.649506,37.126347,"POLYGON ((-2238625.809 108933.090, -2238524.85..."
5393,490805,PURDUE UNIVERSITY NORTHWEST,2200 169TH STREET,HAMMOND,IN,46323,NOT AVAILABLE,(219) 989-2400,1,A,11883,LAKE,18089,USA,611310,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",https://www.pnw.edu/getting-to-pnw/visiting/dr...,2020-07-01,IMAGERY/OTHER,2020-07-01,www.pnw.edu,10473,1410,NOT AVAILABLE,purdueuniversitynorthwest,7.067316e+05,319575.663150,-87.474015,41.580316,"POLYGON ((706669.413 319655.907, 706779.802 31..."
5394,195562,ST. JOSEPH'S COLLEGE-LONG ISLAND,155 W ROE BLVD,PATCHOGUE,NY,11772,NOT AVAILABLE,(631) 687-5100,2,A,4876,SUFFO

In [302]:
merged_data = geo_df.merge(
    master_df,
    left_on = ['preprocessed_name', 'STATE'],
    right_on = ['preprocessed_name', 'State'],
    how="outer",
    indicator=True,
    validate='1:1'
)

In [297]:
geo_df['preprocessed_name'].value_counts().head(50)


columbiacollege                                  4
spartancollegeaeronauticstechnology              3
westminstercollege                               3
unioncollege                                     3
lincolnuniversity                                3
emmanuelcollege                                  2
betheluniversity                                 2
northcentraluniversity                           2
gatewaycommunitycollege                          2
mariontechnicalcollege                           2
stjohnscollege                                   2
bethanycollege                                   2
southwesterncommunitycollege                     2
wheatoncollege                                   2
highlandcommunitycollege                         2
bryanuniversity                                  2
southeasterncommunitycollege                     2
marianuniversity                                 2
kingscollege                                     2
middlesexcommunitycollege      

In [298]:
geo_df[geo_df['preprocessed_name']=='westerntechnicalcollege']


,UNIQUEID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,TELEPHONE,TYPE,STATUS,POPULATION,COUNTY,COUNTYFIPS,COUNTRY,NAICS_CODE,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,WEBSITE,TOT_ENROLL,TOT_EMP,SHELTERID,preprocessed_name,centroid_x_epsg_102005,centroid_y_epsg_102005,centroid_long,centroid_lat,geometry


In [291]:
master_df[master_df['preprocessed_name']=='westerntechnicalcollege']

,State,OPE ID,IPED ID,School Name,Institution Type,# of Nat'l AP Partners,# of Partners,Civic Holiday,MSI,NSLVE,NSLVE\n2014,2016 NSLVE Report,2018 NSLVE Reports,2016 Action Plan,2018 Action Plan,2020 Action Plan,Campus Contact,City,Longitude,Latitude,Pell Grant %,BIPOC %,American Indian or Alaskan Native,Asian,Black or African American,Hispanic or Latinx,NHPI,Two or more Races,SLSV Coalition,ALL IN,AGF,Ask Every Student,Campus Vote Project,Voter Friendly Campus,StudentPIRGs,Aliento Education Fund,Alliance for Youth Organizing,American Democracy Project,Arizona Students' Association,Baltimore Collegetown Network\t\t,Black Girls Vote,Boston Votes Coalition,California Campus Compact,CALPIRG Students,CEEP,Civic Nebraska,Common Cause,Count US IN,Creative Campus Voting Project,Day on Democracy,Democracy Works,Engage Miami,Every Vote Counts,Forward Montana,Georgia Shift,Hillel International,IGNITE National,IA & MN Campus Compact,LeadMN,Loud Light,Maine Students Vote,MARYPIRG,MI Familia Vota,Minnesota Youth Collective,Mississippi Votes,NASPAA,New Era Colorado,NYPIRG,NCAAT,NCPIRG,NC Campus Compact,Ohio Campus Compact,PIRGIM Campus Action,Project Pericles,TurnUp Activism,Up to Us/ Net Impact,VoteRiders,Washington Student Association,Xceleader (Vote HBCU),NVEW 2020,NVRD\n2017,NVRD 2018,NVRD 2020,preprocessed_name
3071,WI,3840.00,240170.00,Western Technical College,"Public, 2 Year",3.0,4.0,0.0,,Yes,NaN,NaN,NaN,NaN,NaN,https://allinchallenge.org/wp-content/uploads/...,NaN,La Crosse,-91.246191,43.815309,52,13.26%,1.21%,3.78%,3.10%,3.13%,0.09%,1.94%,,X,,,X,X,,,,,,,,,,,X,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,westerntechnicalcollege


In [292]:
df_temp = master_df.loc[master_df.duplicated(['preprocessed_name', "State"])]
df_temp_geo = geo_df.loc[~geo_df.duplicated(['preprocessed_name', "STATE"])]
df_temp_geo

,UNIQUEID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,TELEPHONE,TYPE,STATUS,POPULATION,COUNTY,COUNTYFIPS,COUNTRY,NAICS_CODE,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,WEBSITE,TOT_ENROLL,TOT_EMP,SHELTERID,preprocessed_name,centroid_x_epsg_102005,centroid_y_epsg_102005,centroid_long,centroid_lat,geometry
0,105367,OTTAWA UNIVERSITY-PHOENIX,9414 NORTH 25TH AVENUE,PHOENIX,AZ,85021,NOT AVAILABLE,(602) 371-1188,2,A,923,MARICOPA,04013,USA,611310,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",www.ottawa.edu,2018-02-28,IMAGERY/OTHER,2019-01-24,www.ottawa.edu,809,114,NOT AVAILABLE,ottawauniversityphoenix,-1.486844e+06,-470622.131776,-112.113054,33.572125,"POLYGON ((-1486881.628 -470586.100, -1486793.2..."
1,110361,CALIFORNIA BAPTIST UNIVERSITY,8432 MAGNOLIA AVE,RIVERSIDE,CA,92504,3297,(877) 228-3615,2,A,12142,RIVERSIDE,06065,USA,611310,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",https://calbaptist.edu/map,2018-05-10,IMAGERY/OTHER,2019-01-24,www.calbaptist.edu,10486,1656,NOT AVAILABLE,californiabaptistuniversity,-1.960011e+06,-331557.164594,-117.426029,33.927877,"POLYGON ((-1960482.345 -331343.278, -1960078.2..."
2,111391,CALIFORNIA WESTERN SCHOOL OF LAW,225 CEDAR ST,SAN DIEGO,CA,92101,3046,(619) 239-0391,2,A,1021,SAN DIEGO,06073,USA,611310,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",https://www.cwsl.edu,2018-05-11,IMAGERY/OTHER,2019-01-24,https://www.cwsl.edu,802,219,NOT AVAILABLE,californiawesternschoollaw,-1.967108e+06,-467374.141459,-117.162558,32.721731,"POLYGON ((-1967135.570 -467340.979, -1967068.1..."
3,113698,DOMINICAN UNIVERSITY OF CALIFORNIA,50 ACACIA AVE,SAN RAFAEL,CA,94901,2298,(415) 457-4440,2,A,2395,MARIN,06041,USA,611310,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",https://www.dominican.edu/about/campus/mapsand...,2018-08-30,IMAGERY/OTHER,2019-01-24,www.dominican.edu,1779,616,NOT AVAILABLE,dominicanuniversitycalifornia,-2.284703e+06,221228.999296,-122.514107,37.980091,"MULTIPOLYGON (((-2284182.288 221224.380, -2284..."
4,192563,LIU BRENTWOOD,"MICHAEL J. GRANT CAMPUS, SUFFOLK COUNTY COMMUN...",BRENTWOOD,NY,11717,NOT AVAILABLE,(631) 287-8500,2,C,-999,SUFFOLK,36103,USA,611310,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",http://www.liu.edu/Brentwood/About/Visiting/Ca...,2018-12-19,IMAGERY/OTHER,2019-04-11,liu.edu/Brentwood,-999,-999,NOT AVAILABLE,liubrentwood,1.888610e+06,436153.557383,-73.272732,40.797865,"POLYGON ((1888955.082 435997.571, 1888956.510 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5391,11493801,GAVILAN COLLEGE - HOLLISTER SITE,365 FOURTH STREET,HOLLISTER,CA,95023,NOT AVAILABLE,NOT AVAILABLE,1,A,1361,SAN BENITO,06069,USA,611210,JUNIOR COLLEGES,https://www.gavilan.edu/about/hollister.php,2018-12-19,IMAGERY/OTHER,2019-02-12,http://www.gavilan.edu/hollister/index.html,1361,-999,NOT AVAILABLE,gavilancollegehollistersite,-2.225901e+06,73606.022079,-121.402782,36.852230,"POLYGON ((-2225915.052 73641.116, -2225870.144..."
5392,11493802,GAVILAN COLLEGE - MORGAN HILL CAMPUS,17060 MONTEREY ST.,MORGAN HILL,CA,95037,NOT AVAILABLE,NOT AVAILABLE,1,A,1323,SANTA CLARA,06085,USA,611210,JUNIOR COLLEGES,https://www.gavilan.edu/about/morgan_hill.php,2018-12-19,IMAGERY/OTHER,2019-02-12,http://www.gavilan.edu/maps/index.html,1323,-999,NOT AVAILABLE,gavilancollegemorganhillcampus,-2.238508e+06,108881.422727,-121.649506,37.126347,"POLYGON ((-2238625.809 108933.090, -2238524.85..."
5393,490805,PURDUE UNIVERSITY NORTHWEST,2200 169TH STREET,HAMMOND,IN,46323,NOT AVAILABLE,(219) 989-2400,1,A,11883,LAKE,18089,USA,611310,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",https://www.pnw.edu/getting-to-pnw/visiting/dr...,2020-07-01,IMAGERY/OTHER,2020-07-01,www.pnw.edu,10473,1410,NOT AVAILABLE,purdueuniversitynorthwest,7.067316e+05,319575.663150,-87.474015,41.580316,"POLYGON ((706669.413 319655.907, 706779.802 31..."
5394,195562,ST. JOSEPH'S COLLEGE-LONG ISLAND,155 W ROE BLVD,PATCHOGUE,NY,11772,NOT AVAILABLE,(631) 687-5100,2,A,4876,SUFFO

In [184]:
#master_df[master_df['IPED ID']!='Hawaii Community College']
#master_df[master_df['IPED ID']!='485263.00']
#master_df[master_df['IPED ID']!='101949.00']
#master_df[master_df['IPED ID'] != '155636.00']
#geo_df[geo_df['UNIQUEID']!='459082']
#geo_df[geo_df['UNIQUEID']!='241128']



In [129]:
merged_data[merged_data['preprocessed_name']=='columbiacollege']

,UNIQUEID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,TELEPHONE,TYPE,STATUS,POPULATION,COUNTY,COUNTYFIPS,COUNTRY,NAICS_CODE,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,WEBSITE,TOT_ENROLL,TOT_EMP,SHELTERID,preprocessed_name,centroid_x_epsg_102005,centroid_y_epsg_102005,centroid_long,centroid_lat,geometry,State,OPE ID,IPED ID,School Name,Institution Type,# of Nat'l AP Partners,# of Partners,Civic Holiday,MSI,NSLVE,NSLVE\n2014,2016 NSLVE Report,2018 NSLVE Reports,2016 Action Plan,2018 Action Plan,2020 Action Plan,Campus Contact,City,Longitude,Latitude,Pell Grant %,BIPOC %,American Indian or Alaskan Native,Asian,Black or African American,Hispanic or Latinx,NHPI,Two or more Races,SLSV Coalition,ALL IN,AGF,Ask Every Student,Campus Vote Project,Voter Friendly Campus,StudentPIRGs,Aliento Education Fund,Alliance for Youth Organizing,American Democracy Project,Arizona Students' Association,Baltimore Collegetown Network\t\t,Black Girls Vote,Boston Votes Coalition,California Campus Compact,CALPIRG Students,CEEP,Civic Nebraska,Common Cause,Count US IN,Creative Campus Voting Project,Day on Democracy,Democracy Works,Engage Miami,Every Vote Counts,Forward Montana,Georgia Shift,Hillel International,IGNITE National,IA & MN Campus Compact,LeadMN,Loud Light,Maine Students Vote,MARYPIRG,MI Familia Vota,Minnesota Youth Collective,Mississippi Votes,NASPAA,New Era Colorado,NYPIRG,NCAAT,NCPIRG,NC Campus Compact,Ohio Campus Compact,PIRGIM Campus Action,Project Pericles,TurnUp Activism,Up to Us/ Net Impact,VoteRiders,Washington Student Association,Xceleader (Vote HBCU),NVEW 2020,NVRD\n2017,NVRD 2018,NVRD 2020,_merge
1397,217934,COLUMBIA COLLEGE,1301 COLUMBIA COLLEGE DR,COLUMBIA,SC,29203,NOT AVAILABLE,(803) 786-3012,2,A,1527,RICHLAND,45079,USA,611310,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",www.columbiasc.edu/,2018-03-14,IMAGERY/OTHER,2018-03-14,www.columbiasc.edu/,1276,251,NOT AVAILABLE,columbiacollege,1.373733e+06,-436756.701176,-81.030364,34.045950,"POLYGON ((1373738.499 -436454.203, 1373985.559...",SC,3430.00,217934.00,Columbia College,"Private, 4 Year",0.0,0.0,0.0,,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Columbia,-81.029967,34.045581,46,39.10%,0.74%,0.59%,31.97%,3.35%,0.05%,2.39%,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,both
2038,112561,COLUMBIA COLLEGE,11600 COLUMBIA COLLEGE DR,SONORA,CA,95370,NOT AVAILABLE,(209) 588-5100,1,A,2716,TUOLUMNE,06109,USA,611210,JUNIOR COLLEGES,https://www.gocolumbia.edu/maps/default.php,2018-08-31,IMAGERY/OTHER,2018-08-31,www.gocolumbia.edu/,2496,220,NOT AVAILABLE,columbiacollege,-2.104603e+06,175392.048215,-120.387815,38.030035,"POLYGON ((-2104834.610 175803.422, -2104723.28...",CA,7707.00,112561.00,Columbia College,"Public, 2 Year",0.0,0.0,0.0,,Yes,http://www.gocolumbia.edu/institutional_resear...,NaN,NaN,NaN,NaN,NaN,NaN,Sonora,-120.388385,38.030367,30,27.43%,1.72%,1.66%,3.33%,19.94%,0.47%,0.31%,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,both


In [130]:

for i, r in data.iterrows():
    if "MULTIPOLYGON" in str(data.loc[i,'geometry']):
        data.loc[i,'geo_type']='MULTIPOLYGON'
    elif "POLYGON" in str(data.loc[i,'geometry']):
        data.loc[i,'geo_type']='POLYGON'
    elif "POINT" in str(data.loc[i,'geometry']):
        data.loc[i,'geo_type']='POINT'
        



In [131]:
data['geo_type'].value_counts()

POLYGON         1662
MULTIPOLYGON     553
Name: geo_type, dtype: int64

In [132]:
type(data)

geopandas.geodataframe.GeoDataFrame

In [133]:
def string_punc(test_str):

    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

    try:

        test_str=str(test_str).lower()
        #print(test_str)
        for ele in test_str:
            if ele in punc:
                test_str = test_str.replace(ele, "")
        processed_lst=[]
        for word in test_str.split():
            if word not in ['the','at','in','and','of']:
                processed_lst.append(word)

        test_str=' '.join(processed_lst)
        print(test_str)
        return test_str.lower()
    except:
        return 'Null'

In [134]:
data['preprocessed_name'] = data['NAME'].apply(string_punc)

california baptist university
dominican university california
point loma nazarene university
hawaii pacific university
columbia college chicago
jefferson community technical college
north shore community college
georgia state universityperimeter college
blackfeet community college
hillsborough community college
berkeley collegenew york
bevill state community college
arizona western college
jefferson state community college
central arizona college
university south alabama
scottsdale community college
lurleen b wallace community college
allan hancock college
california state universityeast bay
aurora university
berry college
wilmington university
california lutheran university
carl sandburg college
university miami
southwestern community college
northeastern illinois university
indian hills community college
iowa state university
kirkwood community college
southeastern community college
butler community college
kansas city kansas community college
depaul university
iowa central community

In [135]:
#Extracting the centroid from each college polygon
data['centroid']=data.centroid
data = data.assign(x_centroid = lambda x: (x['centroid'].x))
data = data.assign(y_centroid = lambda x: (x['centroid'].y))
data['centroid_geom'] = data['x_centroid'].map(str) + ', ' + data['y_centroid'].map(str)




/var/folders/mz/72pm8_k52zg04yv8wtwty5ym0000gn/T/ipykernel_19590/2525764517.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  data['centroid']=data.centroid


In [136]:
data

,UNIQUEID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,TELEPHONE,TYPE,STATUS,POPULATION,COUNTY,COUNTYFIPS,COUNTRY,NAICS_CODE,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,WEBSITE,TOT_ENROLL,TOT_EMP,SHELTERID,preprocess,centroid_x,centroid_y,centroid_l,centroid_1,State_1,OPE ID,IPED ID,School Nam,Institutio,# of Nat'l,# of Partn,Civic Holi,MSI,NSLVE,NSLVE\n2014,2016 NSLVE,2018 NSLVE,2016 Actio,2018 Actio,2020 Actio,Campus Con,City_1,Longitude,Latitude,Pell Grant,BIPOC %,American I,Asian,Black or A,Hispanic o,NHPI,Two or mor,SLSV Coali,ALL IN,AGF,Ask Every,Campus Vot,Voter Frie,StudentPIR,Aliento Ed,Alliance f,American D,Arizona S,Baltimore,Black Girl,Boston Vot,California,CALPIRG St,CEEP,Civic Nebr,Common Cau,Count US I,Creative C,Day on Dem,Democracy,Engage Mia,Every Vote,Forward Mo,Georgia Sh,Hillel Int,IGNITE Nat,IA & MN Ca,LeadMN,Loud Light,Maine Stud,MARYPIRG,MI Familia,Minnesota,Mississipp,NASPAA,New Era Co,NYPIRG,NCAAT,NCPIRG,NC Campus,Ohio Campu,PIRGIM Cam,Project Pe,TurnUp Act,Up to Us/,VoteRiders,Washington,Xceleader,NVEW 2020,NVRD\n2017,NVRD 2018,NVRD 2020,geometry,geo_type,preprocessed_name,centroid,x_centroid,y_centroid,centroid_geom
0,110361,CALIFORNIA BAPTIST UNIVERSITY,8432 MAGNOLIA AVE,RIVERSIDE,CA,92504,3297,(877) 228-3615,2,A,12142,RIVERSIDE,06065,USA,611310,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",https://calbaptist.edu/map,2018-05-10,IMAGERY/OTHER,2019-01-24,www.calbaptist.edu,10486,1656,NOT AVAILABLE,californiabaptistuniversity,-1.960011e+06,-3.315572e+05,-117.426029,33.927877,CA,1125.00,110361.00,California Baptist University,"Private, 4 Year",0.0,1.0,0.0,HSI,No,None,None,None,None,None,None,None,Riverside,-117.425619,33.929321,46,55.51%,0.57%,5.36%,8.65%,34.58%,0.93%,5.42%,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,X,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"POLYGON ((-1960482.345 -331343.278, -1960078.2...",POLYGON,california baptist university,POINT (-1960010.717 -331557.165),-1.960011e+06,-3.315572e+05,"-1960010.71692041, -331557.1645936489"
1,113698,DOMINICAN UNIVERSITY OF CALIFORNIA,50 ACACIA AVE,SAN RAFAEL,CA,94901,2298,(415) 457-4440,2,A,2395,MARIN,06041,USA,611310,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",https://www.dominican.edu/about/campus/mapsand...,2018-08-30,IMAGERY/OTHER,2019-01-24,www.dominican.edu,1779,616,NOT AVAILABLE,dominicanuniversitycalifornia,-2.284703e+06,2.212290e+05,-122.514107,37.980091,CA,1196.00,113698.00,Dominican University of California,"Private, 4 Year",1.0,2.0,3.0,None,Yes,None,http://www.allinchallenge.org/wp-content/uploa...,https://www.allinchallenge.org/wp-content/uplo...,http://www.allinchallenge.org/wp-content/uploa...,None,None,None,San Rafael,-122.512051,37.981279,30,52.96%,0.57%,22.26%,4.43%,18.55%,1.08%,6.08%,None,X,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,X,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,X,X,X,"MULTIPOLYGON (((-2284182.288 221224.380, -2284...",MULTIPOLYGON,dominican university california,POINT (-2284703.112 221228.999),-2.284703e+06,2.212290e+05,"-2284703.112188862, 221228.99929643606"
2,121309,POINT LOMA NAZARENE UNIVERSITY,3900 LOMALAND DR,SAN DIEGO,CA,92106,2899,(619) 849-2200,2,A,5546,SAN DIEGO,06073,USA,611310,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",https://www.pointloma.edu/sites/default/files/...,2018-10-12,IMAGERY/OTHER,2019-01-24,www.pointloma.edu,4592,954,NOT AVAILABLE,pointlomanazareneuniversity,-1.975381e+06,-4.659709e+05,-117.251845,32.716852,CA,1262.00,121309.00,Point Loma Nazarene University,"Private, 4 Year",0.0,0.0,0.0,None,No,None,None,None,None,None,None,None,San Diego,-117.250265,32.718391,30,45.38%,0.39%,5.96%,3.21%,28.35%,0.98%,6.48%,None,None,None,None,None,None

In [137]:
master_df = pd.read_csv("../00_source_data/College_Data/SLSV Master Campus Sheet - Master Sheet.csv", header=1)
print("Original Master:",len(master_df))
master_df['preprocessed_name'] = master_df['School Name'].apply(string_punc)
master_df=master_df[~master_df['preprocessed_name'].str.contains('policy|law|phd|graduate|grad|health|medicine|medical|music|business|nursing|film|art|arts|design|commerce', regex=True)]
print("Length of Master Table:",len(master_df))


Original Master: 3114
alaska christian college
alaska pacific university
ilisagvik college
prince william sound community college
university alaska anchorage
university alaska fairbanks
university alaska southeast
alabama a m university
alabama state university
amridge university
athens state university
auburn university
auburn university montgomery
bevill state community college
birmingham southern college
bishop state community college
calhoun community college
central alabama community college
chattahoochee valley community college
columbia southern university
concordia college alabama
enterprise state community college
faulkner university
gadsden state community college
george c wallace community college
heritage christian university
herzing university birmingham
huntingdon college
jacksonville state university
jefferson state community college
jf drake state technical college
judson college
lawson state community college
lurleen b wallace community college
marion military institut

In [138]:
final_data=master_df.merge(data, left_on='preprocessed_name', right_on='preprocessed_name',how='inner')




In [139]:
final_data

,State,OPE ID_x,IPED ID_x,School Name,Institution Type,# of Nat'l AP Partners,# of Partners,Civic Holiday,MSI_x,NSLVE_x,NSLVE\n2014_x,2016 NSLVE Report,2018 NSLVE Reports,2016 Action Plan,2018 Action Plan,2020 Action Plan,Campus Contact,City,Longitude_x,Latitude_x,Pell Grant %,BIPOC %_x,American Indian or Alaskan Native,Asian_x,Black or African American,Hispanic or Latinx,NHPI_x,Two or more Races,SLSV Coalition,ALL IN_x,AGF_x,Ask Every Student,Campus Vote Project,Voter Friendly Campus,StudentPIRGs,Aliento Education Fund,Alliance for Youth Organizing,American Democracy Project,Arizona Students' Association,Baltimore Collegetown Network\t\t,Black Girls Vote,Boston Votes Coalition,California Campus Compact,CALPIRG Students,CEEP_x,Civic Nebraska,Common Cause,Count US IN,Creative Campus Voting Project,Day on Democracy,Democracy Works,Engage Miami,Every Vote Counts,Forward Montana,Georgia Shift,Hillel International,IGNITE National,IA & MN Campus Compact,LeadMN_x,Loud Light_x,Maine Students Vote,MARYPIRG_x,MI Familia Vota,Minnesota Youth Collective,Mississippi Votes,NASPAA_x,New Era Colorado,NYPIRG_x,NCAAT_x,NCPIRG_x,NC Campus Compact,Ohio Campus Compact,PIRGIM Campus Action,Project Pericles,TurnUp Activism,Up to Us/ Net Impact,VoteRiders_x,Washington Student Association,Xceleader (Vote HBCU),NVEW 2020_x,NVRD\n2017_x,NVRD 2018_x,NVRD 2020_x,preprocessed_name,UNIQUEID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,TELEPHONE,TYPE,STATUS,POPULATION,COUNTY,COUNTYFIPS,COUNTRY,NAICS_CODE,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,WEBSITE,TOT_ENROLL,TOT_EMP,SHELTERID,preprocess,centroid_x,centroid_y,centroid_l,centroid_1,State_1,OPE ID_y,IPED ID_y,School Nam,Institutio,# of Nat'l,# of Partn,Civic Holi,MSI_y,NSLVE_y,NSLVE\n2014_y,2016 NSLVE,2018 NSLVE,2016 Actio,2018 Actio,2020 Actio,Campus Con,City_1,Longitude_y,Latitude_y,Pell Grant,BIPOC %_y,American I,Asian_y,Black or A,Hispanic o,NHPI_y,Two or mor,SLSV Coali,ALL IN_y,AGF_y,Ask Every,Campus Vot,Voter Frie,StudentPIR,Aliento Ed,Alliance f,American D,Arizona S,Baltimore,Black Girl,Boston Vot,California,CALPIRG St,CEEP_y,Civic Nebr,Common Cau,Count US I,Creative C,Day on Dem,Democracy,Engage Mia,Every Vote,Forward Mo,Georgia Sh,Hillel Int,IGNITE Nat,IA & MN Ca,LeadMN_y,Loud Light_y,Maine Stud,MARYPIRG_y,MI Familia,Minnesota,Mississipp,NASPAA_y,New Era Co,NYPIRG_y,NCAAT_y,NCPIRG_y,NC Campus,Ohio Campu,PIRGIM Cam,Project Pe,TurnUp Act,Up to Us/,VoteRiders_y,Washington,Xceleader,NVEW 2020_y,NVRD\n2017_y,NVRD 2018_y,NVRD 2020_y,geometry,geo_type,centroid,x_centroid,y_centroid,centroid_geom
0,AK,1061.00,102669.00,Alaska Pacific University,"Private, 4 Year",0.0,1.0,1.0,ANNH,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anchorage,-149.804168,61.190968,35,43.07%,20.10%,2.36%,3.89%,5.24%,0.68%,10.81%,,,,,,,,,,,,,,,,,,,,,,,,,,,,X,,,,,,,,,,,,,,,,,,,,,,,,,,,X,alaska pacific university,102669,ALASKA PACIFIC UNIVERSITY,4101 UNIVERSITY DR,ANCHORAGE,AK,99508,NOT AVAILABLE,(907) 561-1266,2,A,650,ANCHORAGE,02020,USA,611310,"COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS",https://www.alaskapacific.edu/visit/campus-map/,2017-11-28,IMAGERY/OTHER,2017-11-28,www.alaskapacific.edu,502,148,NOT AVAILABLE,alaskapacificuniversity,-2.998176e+06,3.379103e+06,-149.804094,61.190323,AK,1061.00,102669.00,Alaska Pacific University,"Private, 4 Year",0.0,1.0,1.0,ANNH,No,None,None,None,None,None,None,None,Anchorage,-149.804168,61.190968,35,43.07%,20.10%,2.36%,3.89%,5.24%,0.68%,10.81%,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,X,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,X,"POLYGON ((-2998337.816 3379241.510, -2998169.9...",POLYGON,POINT (-2998175.532 3379102.981),-2.998176e+06,3.379103e+06,"-2998175.5323420456, 3379102.9814243685"
1,AK,11462.00,102553.00,"University of Alaska, Anchorage","Public, 4 Year",1.0,4.0,2.0,,Yes,NaN,NaN,NaN,NaN,http://www.allinchallenge.org/wp-con

In [140]:
 _=data.to_file("R1.geojson", driver='GeoJSON')

TypeError: Cannot interpret '<geopandas.array.GeometryDtype object at 0x17d1926a0>' as a data type